### .merge()

In [15]:
import pandas as pd

pd.set_option('display.max_columns', 35)
pd.set_option("display.max_colwidth",3000)
pd.set_option("display.max_rows", None)

In [82]:
import pandas as pd
import numpy as np
df1 = pd.DataFrame({'name': ['Fred', 'Suzy', 'Suzy', 'Bob'],
                    'pet': ['Dog', 'Dog', 'Cat', 'Fish']})

df2 = pd.DataFrame({'Name': ['Suzy', 'Suzy', 'Suzy', 'Fred', 'Joe', 'Joe'],
                    'Color': ['Black', 'Blue', 'Red', 'Green', 'Yellow', 'Blue']})


In [32]:
display(df1), display(df2)

,name,pet
0,Fred,Dog
1,Suzy,Dog
2,Suzy,Cat
3,Bob,Fish


,Name,Color
0,Suzy,Black
1,Suzy,Blue
2,Suzy,Red
3,Fred,Green
4,Joe,Yellow
5,Joe,Blue


(None, None)

In [89]:
%%time
(df1.merge(df2, left_on='name', right_on='Name', how='left')
    .assign(pet = df1.pet.fillna("no_pet"),
            name = df1.name.replace("Suzy", 'No_Suzy')))

CPU times: total: 0 ns
Wall time: 3 ms


,name,pet,Name,Color
0,Fred,Dog,Fred,Green
1,No_Suzy,Dog,Suzy,Black
2,No_Suzy,Cat,Suzy,Blue
3,Bob,Fish,Suzy,Red
4,NaN,NaN,Suzy,Black
5,NaN,NaN,Suzy,Blue
6,NaN,NaN,Suzy,Red
7,NaN,NaN,NaN,NaN


In [76]:
%%time
# be careful with names. Only because there was a white space after the column `name `, the code below didn't want to work
# (df1.merge(df2.assign(name=df2.Name) )) # by default it's an inner join

# df1.merge(df2, left_on='name', right_on='Name', how='left')
(df1[['name']].merge(df2[['Name', 'Color']], left_on='name', right_on='Name', how='inner')
              .rename(columns={'Color':'Theme'})
              .drop(columns=['Name']))


CPU times: total: 0 ns
Wall time: 4.01 ms


,name,Theme
0,Fred,Green
1,Suzy,Black
2,Suzy,Blue
3,Suzy,Red
4,Suzy,Black
5,Suzy,Blue
6,Suzy,Red


In [16]:

# !pip install pyarrow fastparquet
df3 = (pd.read_parquet(r"data/9015-2013-11-12.parquet")
        .drop(columns=['accessionNumber', 'cikManager', 'managerName', 'periodOfReport',
        'submissionType', 'filedAsOfDate', 'xml_flag', 'updated_at', 'tr_type', 'tr_shares', 'tr_value',
        'quarter_as_int', 'prev_quarter_diff', 'next_quarter_diff',
        'quarter_max_filed', 'prev_shares_diff', 'prev_value_diff',
        'next_shares_diff', 'next_value_diff']))#.head(1000)
df4 = pd.read_parquet(r"data/cusip_md.parquet")#.head(10000)
df3.head()

,cusip,nameOfIssuer,titleOfClass,value,shares,shrsOrPrnAmt,putCall,quarter
0,Y8565J101,COM,COM,507000.0,15190.0,SH,None,2013Q3
1,Y8564W103,None,None,0.0,0.0,SH,None,2010Q2
2,Y8564W103,None,None,576000.0,25300.0,SH,None,2010Q1
3,Y8564W103,None,None,217000.0,9900.0,SH,None,2009Q3
4,Y8564W103,None,None,162000.0,11426.0,SH,None,2009Q1


In [18]:
df3.columns, df4.columns, df3.shape, df4.shape


(Index(['cusip', 'nameOfIssuer', 'titleOfClass', 'value', 'shares',
        'shrsOrPrnAmt', 'putCall', 'quarter'],
       dtype='object'),
 Index(['quarter', 'cusip', 'ticker', 'cusip_description', 'nameofissuer',
        'titleofclass'],
       dtype='object'),
 (76879, 8),
 (1914747, 6))

In [17]:
%%time
# when merging, if left, the values that exist on the left but not on the right, will get displayed, but for the right df, there will be nan for the common column
import numpy as np

df34 = (df3[['cusip', 'quarter','nameOfIssuer', 'titleOfClass']].merge(df4[['cusip', 'quarter','nameofissuer', 'titleofclass']], \
                                                               left_on=['cusip', 'quarter'], right_on=['cusip', 'quarter'], how='left'))


# df34 = (df34.assign(nameOfIssuer=df34['nameofissuer'],
#                   titleOfClass=df34['titleofclass'])
#             .drop(columns=['nameofissuer', 'titleofclass']))
# df34.titleOfClass[df34.titleOfClass == 'COM']  = 'DOT.COM'
# df34.titleOfClass[df34.quarter == '2010Q2']  = 'no_COM'
# df34["cusip"] = np.where(df34["nameOfIssuer"].str.contains('PARTNER'), "1",  df34.cusip)
df34.head()

CPU times: total: 531 ms
Wall time: 513 ms


,cusip,quarter,nameOfIssuer,titleOfClass,nameofissuer,titleofclass
0,Y8565J101,2013Q3,COM,COM,TEEKAY OFFSHORE PARTNERS L P,PARTNERSHIP UN
1,Y8564W103,2010Q2,None,None,TEEKAY CORPORATION,COM
2,Y8564W103,2010Q1,None,None,TEEKAY CORPORATION,COM
3,Y8564W103,2009Q3,None,None,TEEKAY CORPORATION,COM
4,Y8564W103,2009Q1,None,None,TEEKAY CORPORATION,COM


In [ ]:
# (df3.merge(df4, left_on=['quarter', 'cusip'], right_on=['quarter', 'cusip'], how='left' , indicator=True)) 

In [43]:
url = r"https://raw.githubusercontent.com/mattharrison/datasets/" \
       "master/data/dirtydevil.txt"

df = pd.read_csv(url, skiprows=lambda num: num <34 or num ==35, sep='\t')
df.head()

C:\Users\yo_fanpc\AppData\Local\Temp\ipykernel_20496\3616401469.py:4: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url, skiprows=lambda num: num <34 or num ==35, sep='\t')


,agency_cd,site_no,datetime,tz_cd,144166_00060,144166_00060_cd,144167_00065,144167_00065_cd
0,USGS,9333500,2001-05-07 01:00,MDT,71.0,A:[91],NaN,NaN
1,USGS,9333500,2001-05-07 01:15,MDT,71.0,A:[91],NaN,NaN
2,USGS,9333500,2001-05-07 01:30,MDT,71.0,A:[91],NaN,NaN
3,USGS,9333500,2001-05-07 01:45,MDT,70.0,A:[91],NaN,NaN
4,USGS,9333500,2001-05-07 02:00,MDT,70.0,A:[91],NaN,NaN


In [44]:
def to_us_mountain_time(df_, time_col, tz_col):
    return (df_
            .assign(**{tz_col: df_[tz_col].replace('MDT', 'MST7MDT')})
            .groupby(tz_col)
            [time_col]
            .transform(lambda s: pd.to_datetime(s)
                      .dt.tz_localize(s.name, ambiguous=True)
                      .dt.tz_convert('US/Mountain'))
        )

In [1]:
# !python -m pip install numba vaex
import vaex

In [2]:
df_vaex1 = vaex.open(r"data/9015-2013-11-12.parquet")
df_vaex2 = vaex.open(r"data/cusip_md.parquet")

In [10]:
%%time
# (df3.merge(df4, left_on=['quarter', 'cusip'], right_on=['quarter', 'cusip'], how='left' , indicator=True)) 
df_vaex_result = (df_vaex1[['cusip', 'quarter','nameOfIssuer', 'titleOfClass']]
         .join(df_vaex2[['cusip', 'quarter','nameofissuer', 'titleofclass']],
                left_on="cusip",
                right_on="cusip", how='left', lprefix='y_', allow_duplication=True))

CPU times: total: 1.06 s
Wall time: 319 ms


In [12]:
df_vaex_result.filter('y_cusip' == "Y8564W103")

#,y_cusip,nameOfIssuer,titleOfClass,y_quarter,cusip,quarter,nameofissuer,titleofclass
0,Y8565J101,COM,COM,2013Q3,Y8565J101,2020Q1,TEEKAY OFFSHORE PARTNERS L P,PARTNERSHIP UN
1,Y8564W103,None,None,2010Q2,Y8564W103,2022Q3,None,None
2,Y8564W103,None,None,2010Q1,Y8564W103,2022Q3,None,None
3,Y8564W103,None,None,2009Q3,Y8564W103,2022Q3,None,None
4,Y8564W103,None,None,2009Q1,Y8564W103,2022Q3,None,None
...,...,...,...,...,...,...,...,...
"4,698,664",227116100,None,None,2005Q2,227116100,2000Q3,CROMPTON CORP,COM
"4,698,665",227111101,None,None,1999Q3,227111101,1999Q2,CROMPTON & KNOWLES CORP,COM
"4,698,666",227111101,None,None,1999Q3,227111101,1999Q1,¥ CROMPTON & KNOWLES CORP,COM
"4,698,667",227111101,None,None,1999Q2,227111101,1999Q2,CROMPTON & KNOWLES CORP,COM
